In [1]:
from pygod.utils import load_data
from torch_geometric.data import Data
import sys
import torch
import torch_sparse
from torch_sparse import SparseTensor

sys.path.append('../')
from GAD.DOMINANT.dominant_surrogate import AutoEncoder, Trainer
from Poison.nettack import Nettack

data: Data = load_data("inj_cora")


In [2]:
"""

Setting up a surrogate model.
Due to the 2-layer GCN restriction of Nettack, the architecture of DOMINANT is not
usable as a surrogate model.

Instead, a compatible autoencoder utilising 2 GCN layers for the encoder, and a MLP
for the decoding is used.

"""

def edge_index_to_sparse_tensor(edge_index, num_nodes):
    edge_attr = torch.ones(edge_index.size(1))
    adj = torch_sparse.SparseTensor(row=edge_index[0], col=edge_index[1], value=edge_attr,
                                    sparse_sizes=(num_nodes, num_nodes))
    return adj

input_dim = data.num_node_features
hidden_dim = 32
latent_dim = 16
decoder_hidden_dims = [64, 128, 64]
output_dim = data.num_node_features

sparse_adj = edge_index_to_sparse_tensor(data.edge_index, data.num_nodes) 
sp_adj = sparse_adj.to_scipy(layout="csr")
sp_attr = SparseTensor.from_dense(data.x).to_scipy(layout="csr")
surrogate_model = AutoEncoder(input_dim, hidden_dim, latent_dim, decoder_hidden_dims, output_dim, sp_attr.shape[1], data.y.bool().max().item()+1, hidden_dim)
optimizer = torch.optim.Adam(surrogate_model.parameters(), lr=0.01)
num_epochs = 100

trainer = Trainer(surrogate_model, optimizer, num_epochs)
trainer.train(data.x, data.edge_index, data.y.bool())

# Get the weights of the two GCNConv layers from the encoder.
weights = surrogate_model.get_encoder_weights()


print("WEIGHT SHAPES")
print(weights[0].shape, weights[1].shape)
# Create instance of Nettack class with the surrogate model
#nettack = Nettack(adj=sp_adj, X_obs=sp_attr, z_obs=data.y.bool().detach().cpu().numpy(), W1=weights[0].detach().cpu().numpy(), W2=weights[1].detach().cpu().numpy(), u=0, verbose=True)

print(weights[0].shape, weights[1].shape)

direct_attack = True
n_influencers = 1 if direct_attack else 5
n_perturbations = 10
#n_perturbations = int(degrees[u]) # How many perturbations to perform. Default: Degree of the node
perturb_features = True
perturb_structure = True

#nettack.reset()
#nettack.attack_surrogate(n_perturbations, perturb_structure=perturb_structure, perturb_features=perturb_features, direct=direct_attack, n_influencers=n_influencers)

1433 32 16
Epoch: 0000, AUC-ROC: 0.6294
Epoch [1/100], Loss: 0.0060
Epoch [2/100], Loss: 0.0037
Epoch [3/100], Loss: 0.0023
Epoch [4/100], Loss: 0.0013
Epoch [5/100], Loss: 0.0008
Epoch [6/100], Loss: 0.0005
Epoch [7/100], Loss: 0.0003
Epoch [8/100], Loss: 0.0002
Epoch [9/100], Loss: 0.0002
Epoch [10/100], Loss: 0.0002
Epoch: 0010, AUC-ROC: 0.7141
Epoch [11/100], Loss: 0.0002
Epoch [12/100], Loss: 0.0002
Epoch [13/100], Loss: 0.0002
Epoch [14/100], Loss: 0.0002
Epoch [15/100], Loss: 0.0001
Epoch [16/100], Loss: 0.0001
Epoch [17/100], Loss: 0.0001
Epoch [18/100], Loss: 0.0001
Epoch [19/100], Loss: 0.0001
Epoch [20/100], Loss: 0.0001
Epoch: 0020, AUC-ROC: 0.7095
Epoch [21/100], Loss: 0.0001
Epoch [22/100], Loss: 0.0001
Epoch [23/100], Loss: 0.0001
Epoch [24/100], Loss: 0.0001
Epoch [25/100], Loss: 0.0001
Epoch [26/100], Loss: 0.0001
Epoch [27/100], Loss: 0.0001
Epoch [28/100], Loss: 0.0001
Epoch [29/100], Loss: 0.0001
Epoch [30/100], Loss: 0.0001
Epoch: 0030, AUC-ROC: 0.7129
Epoch [31/10

In [3]:
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.targeted_attack import Nettack
# Setup Surrogate model
# Setup Attack Model
target_node = 0
model = Nettack(surrogate_model, nnodes=sp_adj.shape[0], attack_structure=True, attack_features=True, device='cpu').to('cpu')
# Attack

model.attack(sp_attr, sp_adj, data.y.bool().detach().cpu().numpy(), target_node, n_perturbations=5)
modified_adj = model.modified_adj
modified_features = model.modified_features

AttributeError: 'AutoEncoder' object has no attribute 'nclass'